d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

## Create Tables
Run the cell below to create tables for the questions in this notebook.

In [0]:
%run ../Utilities/05-CreateTables

Declared the following table:
 revenue1

Declared the following table:
 revenue2

Declared the following table:
 revenue3

Declared the following table:
 revenue4

Declared the following tables:
 products

Declared the following table:
 sales

## Question 1: Min Function
### Summary
Compute the minimum value from the **`Amount`** field for each unique value in the **`TrueFalse`** field in the table **`revenue1`**.

### Steps to complete
Write a SQL query that achieves the following: 
* Computes the number of **`true`** and **`false`** records in the **`TrueFalse`** field from the table **`revenue1`**
* Renames the new column to **`count`**
* Store the records in a temporary view named  **`q1Results`** with the following schema:

| column | type |
|--------|--------|
| TrueFalse | boolean |
| MinAmount | int |

A properly completed solution should produce a view similar to this sample output:

|TrueFalse|         count |
|---------|------------------|
|     true|        4956|
|    false|        5044|

In [0]:
%sql
-- TODO  Answer 1

create or replace temporary view q1Results as
select
TrueFalse,
count(TrueFalse) as count
from
revenue1
group by TrueFalse;

select * from q1Results

TrueFalse,count
true,4980
false,5020


## Question 2: Max Function
### Summary
Compute the maximum value from the **`Amount`** field for each unique value in the **`TrueFalse`** field in the table **`revenue2`**.

### Steps to complete
* Computes the maximum **`Amount`** for **`True`** records and **`False`** records from the **`TrueFalse`** field from the table **`revenue2`**
* Renames the new column to **`maxAmount`**
* Store the records in a temporary view named  **`q2Results`** with the following schema:
   
| column | type |
|--------|--------|
| TrueFalse | boolean |
| maxAmount | double |

A properly completed solution should produce a DataFrame similar to this sample output:

|TrueFalse|         MaxAmount|
|---------|------------------|
|     true|        2243937.93|
|    false|2559457.1799999997|

In [0]:
%sql
-- TODO  Answer 2
create or replace temporary view q2Results as
select
TrueFalse,
max(Amount) as MaxAmount
from
revenue2
group by TrueFalse;

select * from q2Results

TrueFalse,MaxAmount
true,9999.15
false,9993.91


## Question 3: Avg Function
### Summary
Compute the average of the **`Amount`** field for each unique value in the **`TrueFalse`** field in the table **`revenue3`**.

### Steps to complete

* Computes the average of **`Amount`** for **`True`** records and **`False`** records from the **`TrueFalse`** field in the table **`revenue3`**.
* Renames the new column to **`avgAmount`**
* Store the records in a temporary view named  **`q3Results`** with the following schema:

| column | type |
|--------|--------|
| TrueFalse | boolean |
| avgAmount | double |

A properly completed solution should produce a DataFrame similar to this sample output:

|TrueFalse|         AvgAmount|
|---------|------------------|
|     true|        2243937.93|
|    false|2559457.1799999997|

In [0]:
%sql
-- TODO  Answer 3

create or replace temporary view q3Results as
select
TrueFalse,
avg(Amount) as avgAmount
from
revenue3
group by TrueFalse;

select * from q3Results

TrueFalse,avgAmount
true,4913.319419678728
false,5027.6703426294835


## Question 4: Pivot
### Summary
Calculate the total **`Amount`** for **`YesNo`** values of **true** and **false** in 2002 and 2003 from the table **`revenue4`**.
    
### Steps to complete
* Casts the **`UTCTime`** field to Timestamp and names the new column **`Date`**
* Extracts a **`Year`** column from the **`Date`** column
* Filters for years greater than 2001 and less than or equal to 2003
* Groups by **`YesNo`** and creates a pivot table to get the total **`Amount`** for each year and each value in **`YesNo`**
* Represents each total amount as a float rounded to two decimal places
* Store the results into a temporary table named **`q4results`**
   
A properly completed solution should produce a view similar to this sample output:

|YesNo|    2002|    2003|
|-----|--------|--------|
| true| 61632.3| 8108.47|
|false|44699.99|35062.22|

In [0]:
%sql
-- TODO  Answer 4

create or replace temporary view q4Results as
select * 
  from (select Year, YesNo, Amount 
    from (select date_format(cast(UTCTime as timestamp), 'y') as Year,
                 YesNo,
                 Amount
          from revenue4)
    where Year > 2001 and Year <= 2003)
  pivot(
    round(sum(Amount), 2) as avg_am
    for Year in (2002, 2003)
    );

select * from q4Results

YesNo,2002,2003
true,1093393.79,1080581.95
false,1001640.51,1028540.38


## Question 5: Null Values and Aggregates
### Summary
Compute sums of **`amount`** grouped by **`aisle`** after dropping null values from **`products`** table.

### Steps to complete

* Drops any rows that contain null values in either the **`itemId`** or the **`aisle`** column
* Aggregates sums of the **`amount`** column grouped by **`aisle`**
* Store the results into a temporary view named  **`q5Results`**

In [0]:
%sql
-- TODO  Answer 5

create or replace temporary view q5Results as
select
  sum(amount),
  aisle
from products
where itemId is not NULL and aisle is not NULL
group by aisle;

select * from q5Results

sum(amount),aisle
63,3
14,5
107,7
56,12
126,2
8,8


## Question 6: Generate Subtotals By Rollup
### Summary
Compute averages of **`income`** grouped by **`itemName`** and **`month`** such that the results include averages across all months as well as a subtotal for an individual month from the **`sales`** table. 

### Steps to complete

* Coalesces null values in the **`month`** column generated by the `ROLLUP` clause
* Store the results into a temporary view named  **`q6Results`**

Your results should look something like this: 

| itemName| month | avgRevenue |
| --------| ----- | ---------- |
| Anim | 10 | 4794.16 |
| Anim | 7 | 5551.31 |
| Anim | All months | 5046.54 |
| Aute | 4 | 4069.51 |
| Aute | 7 | 3479.31 |
| Aute | 8 | 6339.28 |
| Aute | All months |  4489.41 |
| ... | ... | ... |

In [0]:
%sql
--TODO  Answer 6

create or replace temporary view q6Results as
select 
  coalesce(itemName, 'All items') as itemName,
  coalesce(month(date), 'All months') as month,
  avg(revenue) as avgRevenue
from sales
group by rollup(itemName, month(date))
order by itemName, month;

select * from q6Results

itemName,month,avgRevenue
Ad,1,6612.64
Ad,10,5429.535
Ad,2,4280.2
Ad,7,1716.83
Ad,All months,4693.748
Adipiscing,1,2453.71
Adipiscing,12,1349.25
Adipiscing,2,4034.59
Adipiscing,7,8688.46
Adipiscing,All months,4112.12


-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>